In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

# === KeyZeroBERTo ===

# Dependencies

In [37]:
# !pip install transformers
# !pip install keybert==0.5.0
# !pip install datasets
# !pip install flair|
# %env PYTORCH_ENABLE_MPS_FALLBACK=1


In [92]:
%load_ext autoreload
%autoreload 2
import nltk
import ipywidgets
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
tqdm.pandas()
# from pandarallel import pandarallel
# pandarallel.initialize(progress_bar=True)
from transformers import pipeline
import torch
import datetime
import os
import datasets_handler
import evaluation_metrics
import zeroberto
from zeroberto import ZeroBERTo
# def getAgora():
#     return str(datetime.datetime.now().strftime("%Y_%m_%d__%H_%M_%S"))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data Prep

## Source

In [82]:
# which_dataset = 'folhauol' 
# which_dataset = 'bbc-news'
which_dataset = 'ag_news'

hyp_template = "{}."
# hyp_template = "O tema principal deste texto é {}."
# hyp_template = "this text is about {}."
# hyp_template = "this article is about {}."

raw_data, data_col, class_col = datasets_handler.getDataset(which_dataset)


Found cached dataset ag_news (/Users/alealcoforado/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

## Dataset build

### Train Data

In [83]:
train = raw_data
len(train),train.columns

(127600, Index(['index', 'text', 'label', 'class'], dtype='object'))

In [84]:
classes_list = list(train[class_col].unique())
print(classes_list)

['business', 'science and technology', 'sports', 'world']


# Sentence Extraction

In [85]:
tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()

In [86]:
train['sentences'] = train[data_col].progress_apply(lambda txt:tokenizer.tokenize(txt))
train.head(2)   

100%|██████████| 127600/127600 [00:01<00:00, 109701.71it/s]


,index,text,label,class,sentences
0,0,Wall St. Bears Claw Back Into the Black (Reute...,2,business,"[Wall St., Bears Claw Back Into the Black (Reu..."
1,1,Carlyle Looks Toward Commercial Aerospace (Reu...,2,business,[Carlyle Looks Toward Commercial Aerospace (Re...


In [87]:
sentences_to_classify = []
for sentences in train['sentences']:
  try:
    for sentence in sentences:
      sentences_to_classify.append(sentence)
      break
  except:
    print(train['sentences'])
train['sentence_to_zeroshot'] = sentences_to_classify
len(train)


127600

In [88]:
train['len_sentences'] = train['sentences'].progress_apply(lambda txt:len(txt))
# train[['len_sentences']].describe()

100%|██████████| 127600/127600 [00:00<00:00, 1905205.53it/s]


# Zero-Shot

## Parameters

In [93]:
# max_inferences = len(train) #estava em 5000
max_inferences = 3000

# zeroshot_method = "probability_threshold"
probability_goal = 0.9
top_n_goal = 8

zeroshot_method = "dotproduct"  
# zeroshot_method = "kmeans"  


zeroshot_config = {
    'dataset':which_dataset,
    'method':zeroshot_method,
    'prob_goal':probability_goal,
    'top_n_goal':top_n_goal,
    'max_inferences':max_inferences,
    'classes':classes_list,#list(dict_classes_folha.values())
    'template': hyp_template,
    'random_state':422
}
zeroshot_config

{'dataset': 'ag_news',
 'method': 'dotproduct',
 'prob_goal': 0.9,
 'top_n_goal': 8,
 'max_inferences': 3000,
 'classes': ['business', 'science and technology', 'sports', 'world'],
 'template': '{}.',
 'random_state': 422}

## Data

In [94]:
zeroshot_data = train.sample(zeroshot_config['max_inferences'],random_state=zeroshot_config['random_state']).sort_index()
len(zeroshot_data)

3000

## Model

In [95]:
# classifier = pipeline("zero-shot-classification",
#                       model="joeddav/xlm-roberta-large-xnli",device="cpu") # device = 'cuda'  ##X## DESCOMENTAR AQUI 


model = ZeroBERTo(classes_list=zeroshot_config['classes'],hypothesis_template=zeroshot_config['template'],
                  train_dataset=zeroshot_data,labeling_method=zeroshot_config['method'])

## Inference

In [96]:
%%time
# zeroshot_results = zeroberto.runZeroShotPipeline(classifier,zeroshot_data['sentence_to_zeroshot'],zeroshot_config)  ##X## DESCOMENTAR AQUI 

zeroshot_results = zeroberto.runZeroberto(model,zeroshot_data['text'],zeroshot_config)  ##X## COMENTAR AQUI 
# print(len(zeroshot_results))
### 100 predições ---> <240 segundos (4 min)
### 200 predições ---> <480 segundos (8 min)
### 1000 predições ---> <2400 segundos (40 min)
### 5000 predições ---> 12000 segundos (200 min) (3h 20min)

Preds: 50  - Total time: 2.65 seconds - ETA: 2.6 minutes
Preds: 100  - Total time: 5.08 seconds - ETA: 2.5 minutes
Preds: 150  - Total time: 7.61 seconds - ETA: 2.5 minutes
Preds: 200  - Total time: 10.08 seconds - ETA: 2.5 minutes
Preds: 250  - Total time: 12.73 seconds - ETA: 2.5 minutes
Preds: 300  - Total time: 15.46 seconds - ETA: 2.6 minutes
Preds: 350  - Total time: 18.04 seconds - ETA: 2.6 minutes
Preds: 400  - Total time: 20.62 seconds - ETA: 2.6 minutes
Preds: 450  - Total time: 23.2 seconds - ETA: 2.6 minutes
Preds: 500  - Total time: 25.64 seconds - ETA: 2.6 minutes
Preds: 550  - Total time: 29.26 seconds - ETA: 2.7 minutes
Preds: 600  - Total time: 31.8 seconds - ETA: 2.6 minutes
Preds: 650  - Total time: 34.31 seconds - ETA: 2.6 minutes
Preds: 700  - Total time: 37.08 seconds - ETA: 2.6 minutes
Preds: 750  - Total time: 39.53 seconds - ETA: 2.6 minutes
Preds: 800  - Total time: 42.12 seconds - ETA: 2.6 minutes
Preds: 850  - Total time: 44.62 seconds - ETA: 2.6 minutes
Pre

## Metrics

In [97]:
df_results = model.evaluateLabeling(zeroshot_results)

top 1: {'accuracy': 0.5}
top 2: {'accuracy': 0.75}
top 3: {'accuracy': 0.75}
top 4: {'accuracy': 0.6875}
top 5: {'accuracy': 0.75}
top 6: {'accuracy': 0.7916666666666666}
top 7: {'accuracy': 0.7857142857142857}
top 8: {'accuracy': 0.78125}
top 9: {'accuracy': 0.8055555555555556}
top 10: {'accuracy': 0.8}
top 11: {'accuracy': 0.7954545454545454}
top 12: {'accuracy': 0.7916666666666666}
top 13: {'accuracy': 0.7884615384615384}
top 14: {'accuracy': 0.7857142857142857}
top 15: {'accuracy': 0.7833333333333333}
top 16: {'accuracy': 0.78125}
top 3000: {'accuracy': 0.4786666666666667}


In [98]:
all_metrics = evaluation_metrics.get_metrics(df_results['prediction_code'].to_list(),df_results['class_code'].to_list())
print(all_metrics)

{'weighted': [{'accuracy': 0.4786666666666667}, {'precision': 0.5509133186247495}, {'recall': 0.4786666666666667}, {'f1': 0.4481323685826511}], 'macro': [{'accuracy': 0.4786666666666667}, {'precision': 0.552580706476327}, {'recall': 0.4757023029027709}, {'f1': 0.44726791966399004}]}


{'weighted': [{'accuracy': 0.5213333333333333}, {'precision': 0.5416025863977662}, {'recall': 0.5213333333333333}, {'f1': 0.5142306464390158}], 'macro': [{'accuracy': 0.5213333333333333}, {'precision': 0.5425501838951864}, {'recall': 0.5204099856100467}, {'f1': 0.5143875014658904}]}


## Save Results

In [99]:
save_time = evaluation_metrics.saveZeroshotResults(zeroshot_config,df_results)

predictions_and_probabilities_test_2023_02_17__04_03_23.csv
zeroshot_config_test_2023_02_17__04_03_23.csv


# Load Previous Results

In [100]:
# df_test_results = pd.read_csv(r'/Users/alealcoforado/Documents/Projetos/Datasets/predictions_and_probabilities_test1.csv')

In [101]:
# df_test_results.index = df_test_results['Unnamed: 0']
# df_test_results.drop(columns="Unnamed: 0",inplace=True)

# Step 2: Post Labeling

In [103]:
use_zeroshot_previous_step = True
top_n = 16
exec_time = save_time

In [104]:
if use_zeroshot_previous_step == True:
    zeroshot_previous_data = datasets_handler.getZeroshotPreviousData(which_dataset,class_col,top_n=top_n,exec_time=exec_time)
    raw_data_final, new_class_col = datasets_handler.mergeLabelingToDataset(raw_data,zeroshot_previous_data,class_col)

### Kmeans

In [105]:
model_kmeans = ZeroBERTo(classes_list=zeroshot_config['classes'],hypothesis_template=zeroshot_config['template'],
                  train_dataset=zeroshot_data,labeling_method=zeroshot_config['method'])

In [106]:
df_top_n = raw_data_final.sort_values('top_probability',ascending=False).groupby('prediction_code').head(1)
df_top_n[['text','label','class','prediction_code']]

,text,label,class,prediction_code
35431,Mass. colleges awarded grant for nanotech The ...,2,business,1.0
64703,Kmart names new chief executive l Education: H...,2,business,0.0
7126,Football action that #39;s worth tackling For ...,3,science and technology,2.0
126414,A threat to the West Bank pullout The expected...,0,world,3.0


In [107]:
cluster_centroids = model_kmeans.encode(df_top_n['text'].to_list())

In [108]:
model_kmeans.clusterModel.fit(cluster_centroids)

KMeans(init=array([[ 0.0170192 ,  0.03357903,  0.04584389, ..., -0.03651461,
        -0.03750433, -0.01130612],
       [-0.00778108, -0.00613849,  0.07736567, ..., -0.03057895,
        -0.01588916, -0.00433709],
       [ 0.07192959, -0.00655643,  0.08293311, ...,  0.03691668,
        -0.01099741,  0.01227635],
       [-0.00066345, -0.01935441,  0.09914628, ..., -0.00937063,
         0.00106889, -0.02356128]]),
       max_iter=600, n_clusters=4, n_init=1, random_state=422)

In [109]:
X = model_kmeans.train_dataset[data_col].to_list()
X_emb = model.encode(X)

In [110]:
Y_class =  model_kmeans.train_dataset[class_col].to_list()
Y_code =  model_kmeans.train_dataset['label'].to_list()
pd.Series(zip(Y_class,Y_code)).drop_duplicates()

0                  (business, 2)
3    (science and technology, 3)
7                    (sports, 1)
8                     (world, 0)
dtype: object

In [111]:
y_pred = model_kmeans.clusterModel.predict(X_emb)

In [112]:
try: print(all_metrics)
except: pass
all_metrics = evaluation_metrics.get_metrics(y_pred,Y_code)
print(all_metrics)

{'weighted': [{'accuracy': 0.4786666666666667}, {'precision': 0.5509133186247495}, {'recall': 0.4786666666666667}, {'f1': 0.4481323685826511}], 'macro': [{'accuracy': 0.4786666666666667}, {'precision': 0.552580706476327}, {'recall': 0.4757023029027709}, {'f1': 0.44726791966399004}]}
{'weighted': [{'accuracy': 0.15766666666666668}, {'precision': 0.16257346909893317}, {'recall': 0.15766666666666668}, {'f1': 0.1525905638646822}], 'macro': [{'accuracy': 0.15766666666666668}, {'precision': 0.16268316521437362}, {'recall': 0.15755659734888403}, {'f1': 0.15262188082755315}]}


: 